In [5]:
PORT = 5000
env = 'ICL' # LOCAL | ICL

In [6]:
from pathlib import Path
import os

if env == 'LOCAL':
    WORK_DIR = Path('.')
elif env == 'ICL':
    import os, sys
    WORK_DIR = Path('/vol/bitbucket/') / os.getenv("USER") / 'worker'
    # !pip install -r {WORK_DIR}/requirements.txt

    if not WORK_DIR in sys.path:
        sys.path.append(str(WORK_DIR))

os.environ['WORK_DIR'] = str(WORK_DIR)

In [7]:
from src.analysis.model import MODEL
MODEL.load()

Unrecognized keys in `rope_scaling` for 'rope_type'='linear': {'type'}


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at /vol/bitbucket/kza23/msc/lib/python3.10/site-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/vol/bitbucket/kza23/msc/lib/python3.10/site-packages/uvicorn/main.py", line 577, in run
    server.run()
  File "/vol/bitbucket/kza23/msc/lib/python3.10/site-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/vol/bitbucket/kza23/msc/lib/python3.10/site-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/vol/bitbucket/kza23/msc/lib/python3.10/site-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/vol/bitbucket/kza23/msc/lib/python3.10/site-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._c

In [8]:
import uvicorn
import nest_asyncio
from fastapi import FastAPI, WebSocket, BackgroundTasks
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
from src.constants import UI_URL, WORKER_URL
from src.socket_service import SocketService
from src.work import process_work
from src.models.requests import ProcessRequest

nest_asyncio.apply()

app = FastAPI()

origins = [
    str(UI_URL),
    str(WORKER_URL),
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

work = None

@app.post("/process")
async def process(request: ProcessRequest, background_tasks: BackgroundTasks):
    SocketService.register_incoming_id(request.socket)
    global work
    work = request.work
    # background_tasks.add_task(process_work, request.work)
    return JSONResponse(content={"status": 200})

@app.websocket("/ws/{socket_id}")
async def websocket_endpoint(websocket: WebSocket, socket_id: str, background_tasks: BackgroundTasks):
    async with SocketService(websocket, socket_id) as connection:
        await connection.listen()
        background_tasks.add_task(process_work, work)


if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=PORT)

INFO:     Started server process [2021253]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:5000 (Press CTRL+C to quit)


INFO:     127.0.0.1:60684 - "POST /process HTTP/1.1" 200 OK


INFO:     ('127.0.0.1', 60696) - "WebSocket /ws/6e1de7e0-09ee-41f8-9cf4-ec9895cb5976" [accepted]
INFO:     connection open


Connection accepted!


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Starting tree


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
